In this notbook I am trying to reproduce Zeighami *et al*, 2015.

In [2]:
import livingpark_utils

utils = livingpark_utils.LivingParkUtils()
utils.notebook_init()
# utils.download_ppmi_metadata(["Demographics.csv",
#                             "Primary_Clinical_Diagnosis.csv",
#                             "Cognitive_Categorization.csv",
#                             "Medical_Conditions_Log.csv",
#                             "Concomitant_Medication_Log.csv",
#                             "MDS-UPDRS Part III ON/OFF Determination & Dosing",
#                             "MDS UPDRS Part III",
#                             "Montreal Cognitive Assessment (MoCA)"],headless=False)
# utils.find_nifti_file_in_cache(x["PATNO"], x["EVENT_ID"], x["Description"])
# utils.disease_duration()
# utils.moca2mmse(2)

removing link inputs
removing link outputs
Installing notebook dependencies (see log in install.log)... 
This notebook was run on 2022-11-03 17:49:44 UTC +0000


In [3]:
import re
from functools import reduce
import numpy as np
import pandas as pd

In [4]:
from livingpark_utils.zeighamietal.constants import (
    FILENAME_PARTICIPANT_STATUS,
    FILENAME_DEMOGRAPHICS,
    FILENAME_AGE,
    FILENAME_MOCA,
    FILENAME_UPDRS3,
    FILENAME_T1_INFO,
)

from livingpark_utils.zeighamietal.constants import (
    COL_PAT_ID,
    COL_STATUS,
    COL_VISIT_TYPE,
    COL_DATE_INFO,
)

from livingpark_utils.zeighamietal.constants import (
    STATUS_PD,
    STATUS_HC,
    MAIN_COHORT,
    VISIT_BASELINE,
    VISIT_SCREENING,
    SEX_FEMALE,
    SEX_MALE,
    MAX_DATES,
)

from livingpark_utils.zeighamietal.constants import (
    COL_PD_STATE,
    COL_AGE,
    COL_SEX,
    COL_EDUCATION,
    COL_MOCA,
    COL_PIGD,
    COL_GCO,
    COLS_PIGD_COMPONENTS_UPDRS3,
    COLS_PIGD_COMPONENTS,
    COLS_SCORES,
    COLS_SCORES_WITHOUT_GCO,
    COL_FOLLOWUP,
)

from livingpark_utils.zeighamietal import (
    load_ppmi_csv,
    get_t1_cohort,
    mean_impute,
    filter_date,
)

In [5]:
COL_UPDRS3 = ["NHY", "NP3TOT"]

In [6]:
# from livingpark_utils.scripts import run

# run.mri_metadata()
# run.pd_status()

In [7]:

FILENAME_DEMOGRAPHICS = "Demographics.csv"
FILENAME_AGE = "Age_at_visit.csv"
FILENAME_PARTICIPANT_STATUS = "Participant_Status.csv"
FILENAME_MOCA = "Montreal_Cognitive_Assessment__MoCA_.csv"
FILENAME_UPDRS3 = "MDS_UPDRS_Part_III.csv"

In [8]:
df_status = load_ppmi_csv(utils, FILENAME_PARTICIPANT_STATUS)

cohort_t1_map = {}
cohort_name = MAIN_COHORT

print(f"=============== {cohort_name.capitalize()} cohort ===============")

df_t1_subset = get_t1_cohort(
    utils,
    cohort_name=cohort_name,
    filename=FILENAME_T1_INFO,
    sagittal_only=True,
)
cohort_t1_map[cohort_name] = df_t1_subset

# cohort composition: number of PD patients/healthy controls
print(
    df_status.loc[
        df_status[COL_PAT_ID].isin(df_t1_subset[COL_PAT_ID]), COL_STATUS
    ].value_counts()
)

=============== Main cohort ===============
Removing extra scans for 1 subjects
Parkinson's Disease    236
Healthy Control        113
Name: COHORT_DEFINITION, dtype: int64


In [9]:
cols_for_merge = [COL_PAT_ID, COL_DATE_INFO, COL_VISIT_TYPE]
df_updrs3 = load_ppmi_csv(
    utils, FILENAME_UPDRS3, cols_to_impute=COLS_PIGD_COMPONENTS_UPDRS3 + COL_UPDRS3
)
df_moca = load_ppmi_csv(utils, FILENAME_MOCA)  # do not impute
df_updrs3 = df_updrs3.loc[df_updrs3[COL_PD_STATE] != "OFF"]

df_updrs3 = df_updrs3.loc[:, cols_for_merge + COL_UPDRS3]
df_moca = df_moca.loc[:, cols_for_merge + [COL_MOCA]]

df_assessments_all = reduce(
    lambda df1, df2: df1.merge(df2, on=cols_for_merge, how="outer"),
    [ df_updrs3, df_moca],
).drop_duplicates()

# some missing values remain even if we use the screening visit score
# we will impute these using the original mean
mean_moca = df_moca[COL_MOCA].mean()

cols_to_impute = [col for col in ["NHY", "NP3TOT", "MCATOT"] if col != COL_MOCA]
df_assessments_all = mean_impute(df_assessments_all, cols_to_impute)

# keep only subjects who have a T1
cohort_assessments_map_orig = {}
for cohort_name, df_t1_subset in cohort_t1_map.items():
    cohort_assessments_map_orig[cohort_name] = df_assessments_all.loc[
        df_assessments_all[COL_PAT_ID].isin(df_t1_subset[COL_PAT_ID])
    ]

In [10]:
df_updrs3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16130 entries, 0 to 19365
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   PATNO     16130 non-null  int64         
 1   INFODT    16130 non-null  datetime64[ns]
 2   EVENT_ID  16130 non-null  object        
 3   NHY       16130 non-null  float64       
 4   NP3TOT    16130 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 756.1+ KB


In [11]:
col_date_diff = "date_diff"

cohort_assessments_map = {}
for cohort_name in cohort_assessments_map_orig:

    print(f"========== {cohort_name.upper()} COHORT ==========")

    date_diffs = []

    df_assessments_cohort: pd.DataFrame = cohort_assessments_map_orig[cohort_name]
    df_assessments_baseline = df_assessments_cohort.loc[
        df_assessments_cohort[COL_VISIT_TYPE] == VISIT_BASELINE
    ]
    df_assessments_screening = df_assessments_cohort.loc[
        df_assessments_cohort[COL_VISIT_TYPE] == VISIT_SCREENING
    ]

    # try to fill in missing baseline data
    for idx_row_baseline, row_baseline in df_assessments_baseline.iterrows():

        subject = row_baseline[COL_PAT_ID]
        date_baseline = row_baseline[COL_DATE_INFO]

        # for each score columns
        for col in [COL_MOCA]:
         
            # fill missing values with screening data
            if pd.isna(row_baseline[col]):

                df_screening_subject = df_assessments_screening.loc[
                    df_assessments_screening[COL_PAT_ID] == subject
                ]

                # some subjects in validation set had multiple screening visits
                # in this case we sort them by how close they are to the baseline visit
                n_screening = len(df_screening_subject)
                if n_screening > 1:
                    df_screening_subject[col_date_diff] = (
                        date_baseline - df_screening_subject[COL_DATE_INFO]
                    )
                    df_screening_subject = df_screening_subject.sort_values(
                        col_date_diff, ascending=True
                    )

                # find corresponding assessment score in screening visits
                for idx_row_screening, row_screening in df_screening_subject.iterrows():
                    new_value = row_screening[col]
                    date_diff = date_baseline - row_screening[COL_DATE_INFO]
                    if not pd.isna(new_value):
                        break

                # replace
                if not pd.isna(new_value):
                    df_assessments_baseline.loc[idx_row_baseline, col] = new_value
                    date_diffs.append(date_diff.days)  # for plotting
                    
    subjects_common = set(df_assessments_cohort[COL_PAT_ID])
    # print cohort composition
    print(
        df_status.loc[
            df_status[COL_PAT_ID].isin(subjects_common), COL_STATUS
        ].value_counts()
    )
    
    df_assessments_baseline[COL_FOLLOWUP] = False

    # impute remaining missing MoCA values
    df_assessments_baseline.loc[
        df_assessments_baseline[COL_MOCA].isna(), COL_MOCA
    ] = mean_moca

    cohort_assessments_map[cohort_name] = df_assessments_baseline



========== MAIN COHORT ==========
Parkinson's Disease    236
Healthy Control        113
Name: COHORT_DEFINITION, dtype: int64


In [12]:
for cohort_name, df_assessments in cohort_assessments_map.items():
    df = df_assessments[['NP3TOT', 'MCATOT','NHY']].copy()
    gco_means = df.mean()
    gco_stds = df.std()
    gco = ((df - gco_means) / gco_stds).mean(
        axis="columns"
    )
    cohort_assessments_map[cohort_name][COL_GCO] = gco

In [13]:
# def GCO_calculator(cohort_assessments_map, cols) ->:
#     df = df_assessments[cols].copy()
#     gco_means = df.mean()
#     gco_stds = df.std()
#     gco = ((df - gco_means) / gco_stds).mean(
#         axis="columns"
#     )
#     cohort_assessments_map[cohort_name][COL_GCO] = gco


In [15]:
def to_1_decimal_str(f):
    return str(round(f, 1))


df_age = load_ppmi_csv(utils, FILENAME_AGE)
df_demographics = load_ppmi_csv(utils, FILENAME_DEMOGRAPHICS)

col_male = "is_male"
col_cohort = "cohort"

dfs_summary = []
df_assessments: pd.DataFrame
for cohort_name, df_assessments in cohort_assessments_map.items():

    subjects = df_assessments[COL_PAT_ID].drop_duplicates()

    # general demographics (baseline session only)
    df_assessments =df_assessments.merge(df_status,on=[COL_PAT_ID],how="outer")
    df_summary = df_assessments.merge(df_age, on=[COL_PAT_ID, COL_VISIT_TYPE],how="outer")
    df_demographics[col_male] = (df_demographics[COL_SEX] == SEX_MALE).apply(
        lambda v: 100 if v else 0
    )

   
    df_summary = df_summary.merge(df_demographics, on=COL_PAT_ID,how="outer")
    df_summary = df_summary[[COL_PAT_ID, COL_AGE, col_male, COL_STATUS, COL_UPDRS3[1],COL_UPDRS3[0],COL_MOCA, COL_GCO]]

    # append
    df_summary[col_cohort] = cohort_name
    dfs_summary.append(df_summary)

df_summary = pd.concat(dfs_summary)
df_summary = df_summary.iloc[np.where(df_summary[COL_STATUS].isin([STATUS_PD,STATUS_HC]))]
df_summary = df_summary.drop(columns=COL_PAT_ID)
df_summary_means = (
    df_summary.groupby([COL_STATUS]).mean().applymap(to_1_decimal_str))

df_summary_stds = (
    df_summary.groupby([COL_STATUS]).std().applymap(to_1_decimal_str)
)
df_summary_stds = " (" + df_summary_stds + ")"
df_summary_stds.loc[:, col_male] = ""
df_summary_combined = (df_summary_means + df_summary_stds).T
df_summary_combined = df_summary_combined.applymap(lambda x: "-" if "nan" in x else x)
df_summary_combined = df_summary_combined.rename(
    index={
        COL_AGE: "Age",
        col_male: "Male (%)",
        COL_UPDRS3[1]: "UPDRS Part III",
        COL_UPDRS3[0]: "H & Y",
        COL_MOCA: "MoCA",
        COL_GCO: "Global composite outcome",
    }
)
df_summary_combined


COHORT_DEFINITION        Healthy Control Parkinson's Disease
Age                          60.1 (11.3)          61.2 (9.3)
Male (%)                            62.8                60.9
UPDRS Part III                 1.2 (2.7)          21.5 (8.9)
H & Y                          0.0 (0.1)           1.6 (0.5)
MoCA                          28.3 (1.2)          27.4 (2.2)
Global composite outcome      -0.7 (0.2)           0.3 (0.5)

In [24]:
# df = df_summary.iloc[np.where(df_summary[COL_STATUS] == STATUS_HC)].copy()

In [ ]:
# df.iloc[np.where((~df[COL_UPDRS3[1]].isna()) & (df[COL_UPDRS3[1]] >=0.1))]

In [18]:
len(df.iloc[np.where((~df[COL_UPDRS3[1]].isna()) & (df[COL_UPDRS3[1]] >=0.1))])

41